In [192]:
from urllib.request import Request, urlopen, HTTPError
import urllib.parse
from bs4 import BeautifulSoup
from lxml import etree

In [193]:
# function to trick the website into thinking the spider is user-agent

def get_url_data(url = ""):
    try:
        request = Request(url, headers = {'User-Agent' :\
            "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36"})
        response = urlopen(request)
        data = response.read().decode("utf8")
        return data
    except HTTPError:
        return None

In [194]:
# filtering out the bad URLs
visited = []

with open("python-crawl/crawled_urls.txt", "r") as f:
    lines = f.readlines()

with open("python-crawl/crawled_urls_filtered.txt", "w") as wr_f:
    for line in lines:
        temp = line
        
        if "atom-feed" in temp: # de-duplication
            continue
            
        if "comment_id" in temp or "comments_open" in temp: # forum discussions
            continue
            
        if (temp.startswith("https://aeon.co/essays/") or temp.startswith("https://aeon.co/ideas/") or temp.startswith("https://aeon.co/amp/ideas")or temp.startswith("https://aeon.co/amp/essays")) and ("-" in temp):
            temp = temp.replace("https://aeon.co/amp/ideas/", "")
            temp = temp.replace("https://aeon.co/amp/essays/", "")
            temp = temp.replace("https://aeon.co/essays/", "")
            temp = temp.replace("https://aeon.co/ideas/", "")

            if temp not in visited:
                wr_f.write(line)
                visited.append(temp)

# initial - 9600 links
# filtered - 1600 links

In [195]:
url_file = open("python-crawl/crawled_urls_filtered.txt", "r")

root = etree.Element("data")

for url in url_file:
        # downloading data
        data = get_url_data(url)
        
        if data != None:
            # extracting data from HTML
            soup = BeautifulSoup(data, "html")
            title = soup.title.text
            title = title.replace(" | Aeon Essays", "")
            title = title.replace(" | Aeon Ideas", "")
            
            article_body = soup.find("div", attrs = {"class":"has-dropcap"}).text

            categories = soup.find("div" , attrs={"class":"topics-banner"}).text

            authors = soup.find("p", attrs={"class":"article__body__author-name vcard"}).text

            date = soup.find("time", attrs={"class":"article__date published"}).text
            date = date.replace("\n", "")
            
            # creating an XML file
            article = etree.Element("article")
            
            article_title = etree.Element("title") 
            article_authors = etree.Element("authors")
            article_date = etree.Element("date")
            article_category = etree.Element("category")
            article_text = etree.Element("text")
            
            article_title.text = title
            article_authors.text = authors
            article_date.text = date
            article_category.text = categories
            article_text.text = article_body
            
            article.append(article_title)
            article.append(article_authors)
            article.append(article_date)
            article.append(article_category)
            article.append(article_text)
            
            root.append(article)

writing_file = open("python-crawl/Aeon-corpus.xml", "w")
writing_file.write(etree.tostring(root,encoding=str, pretty_print=True))


41666114